In [1]:
# futures_grid_bot_sim.py
# Full simulation pipeline — uses live data from Binance via ccxt, but DOES NOT place real trades.
# It will open LONG grids when model predicts uptrend and SHORT grids when model predicts downtrend.
# No leverage is used (placeholder comments show where to add leverage later).

import ccxt
import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
import ta
import talib
import uuid
import time
from datetime import datetime, timedelta, timezone


In [2]:
# -------------------------
# CONFIG
# -------------------------
symbol = "BTC/USDT"
market_type = "futures"        # we fetch futures-market data (via ccxt options)
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

FEATURES = [
    'rsi', 'ema12', 'ema26', 'macd', 'signal', 'histogram',
    'dema9', 'sma3', 'tsi', '%k', '%d',
    'atr', 'obv', 'adx', 'cci',
    'return_1', 'roll_mean_5', 'roll_std_5'
]

SAVED_MODEL = "greg_tech_12.pth"
SCALER_FILE = "scaler_2_15m.pkl"


user_grid_spacing = 100
min_tp_pct = 0.01
atr_period = 14
atr_multiplier = 0.1
position_fraction = 0.05
reduced_fraction = 0.5
grid_levels = 16
grid_spacing_atr_factor = 0.1
max_total_exposure = 0.7

prediction_interval = 15 * 60
check_interval_seconds = 1
simulate_only = True  # simulation mode — no real trades
hold_factor = 0.2

# Forced cutoff parameters for frozen grids
frozen_timeout_seconds = 60 * 60
frozen_max_drawdown_pct = 0.2

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# -------------------------
# MODEL (your LSTM)
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

input_dim = len(FEATURES)
hidden_dim = 128
num_layers = 4
output_dim = 3

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.eval()
scaler = joblib.load(SCALER_FILE)

/home/gregory/Trading_bot/venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# -------------------------
# CCXT EXCHANGE SETUP (Futures data)
# -------------------------
exchange = ccxt.binance({
    'apiKey': 'eOUQOwsdCjiNMUDbNsrTV0TUPPF7rAiEXiWtk5CMEGClviPoqBqFhm1KFddUtyoE',
    'secret': 'rpPI79M1aoh8sc7oeZJTj8Fox3ppcBPhtBvWXtTBCWW65q4Ys23tBxo5MZ8tgIZv',
    'options': {
        'defaultType': 'future',  # Use Futures endpoints
    },
    'urls': {
        'api': {
            'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
            'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1',
        }
    }
})
exchange.enable_demo_trading(True)  # Enable demo trading mode


In [5]:
# Helper: fetch minimums / step sizes for rounding
MARKET_INFO = {}
def load_market_info():
    global MARKET_INFO
    markets = exchange.load_markets()
    if symbol in markets:
        info = markets[symbol]
        MARKET_INFO['precision'] = info.get('precision', {})
        MARKET_INFO['limits'] = info.get('limits', {})
    else:
        MARKET_INFO['precision'] = {}
        MARKET_INFO['limits'] = {}


In [6]:
# -------------------------
# INDICATORS (same compute_indicators but lowercase columns)
# -------------------------
def compute_indicators(df):
    df = df.copy()
    df.columns = [c.lower() for c in df.columns]

    out = {}
    # Core
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    # DEMA
    try:
        out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    except Exception:
        out["DEMA9"] = df["close"].ewm(span=9, adjust=False).mean()
    # SMA
    out["SMA3"] = ta.trend.sma_indicator(df["close"], window=3)
    # TSI
    def compute_tsi(close, r1=25, r2=13):
        delta = close.diff()
        ema1 = delta.ewm(span=r1, adjust=False).mean()
        ema2 = ema1.ewm(span=r2, adjust=False).mean()
        abs_delta = delta.abs()
        abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
        abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()
        tsi = 100 * (ema2 / (abs_ema2.replace(0, np.nan)))
        return tsi.fillna(0)
    out["TSI"] = compute_tsi(df["close"])
    # Stochastic
    period, smooth_k, smooth_d = 14, 3, 3
    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()
    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low + 1e-8)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    # Advanced
    out["ATR"] = ta.volatility.AverageTrueRange(df["high"], df["low"], df["close"], window=14).average_true_range()
    out["OBV"] = ta.volume.OnBalanceVolumeIndicator(df["close"], df["volume"]).on_balance_volume()
    out["ADX"] = ta.trend.ADXIndicator(df["high"], df["low"], df["close"], window=14).adx()
    out["CCI"] = ta.trend.cci(df["high"], df["low"], df["close"], window=20)
    # Price derived
    out["Return_1"] = df["close"].pct_change()
    out["Roll_Mean_5"] = df["close"].rolling(5).mean()
    out["Roll_Std_5"] = df["close"].rolling(5).std()

    feat_df = pd.DataFrame(out, index=df.index)
    feat_df = feat_df.dropna().reset_index(drop=True)
    feat_df.columns = [c.lower() for c in feat_df.columns]
    return feat_df

In [7]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 5)
    df['prev_close'] = df['close'].shift(1)
    df['TR'] = df.apply(lambda r: max(r['high'] - r['low'],
                                      abs(r['high'] - r['prev_close']),
                                      abs(r['low'] - r['prev_close'])), axis=1)
    df['ATR'] = df['TR'].rolling(atr_period).mean()
    return float(df['ATR'].iloc[-1])


In [8]:
# -------------------------
# Prediction
# -------------------------
def predict_label_from_window(window_features):
    # window_features: numpy array shape (window_size, features)
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    
    # No manual biasing — purely model-based prediction
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    
    return pred  # 0 = down, 1 = hold, 2 = up


In [9]:
# -------------------------
# Trading state & helpers
# -------------------------
from decimal import Decimal, ROUND_DOWN


open_positions = []   # each: {id, side: 'long'/'short', entry, qty, notional(size_usd), grid_id, created_at, tp_price, sl_price, last_price}
open_orders = []      # each: {id, grid_id, side, price, size_usd, status, ccxt_id, tp, sl}
grids = {}            # grid metadata
blocked_zones = [] 
has_open_position_zones = set()  
profit_drop_blocked_zones = []  # [{'grid_id': ..., 'price': ..., 'side': 'long', 'blocked_at': ..., 'active': True}]
tp_reversal_blocked_zones = []
recent_tp_zones = []  # [{'grid_id': ..., 'tp_price': ..., 'side': 'long'/'short', 'active': True}]
# --- Binance Futures Fee Configuration ---
FEE_MODE = "flat"          # or "rate"
FLAT_FEE_USDT = 0.09       # total (open + close)
TRADING_FEE_RATE = 0.00045 # ~0.045% per side, used if FEE_MODE == "rate"



def usd_to_qty(usd, price):
    return usd / price

def round_amount_qty(qty, symbol_local=None):
    
    """Round quantity according to market precision and ensure it's >= exchange min amount.
    Returns a float suitable for order placement."""
    
    try:
        symbol_send = symbol_local if symbol_local else symbol
        market = exchange.market(symbol_send)
        
        # Get amount precision
        prec = int(market.get('precision', {}).get('amount', 6))
        
        # Get min amount
        min_amount = float(market.get('limits', {}).get('amount', {}).get('min', 10**(-prec)))
        
        # Round down with Decimal
        quant = Decimal(str(qty)).quantize(Decimal('1e-{}'.format(prec)), rounding=ROUND_DOWN)
        qty_rounded = max(Decimal(str(min_amount)), quant)
        
        # Safety: check if rounded qty is valid
        if qty_rounded < Decimal(str(min_amount)):
            print(f"❌ Qty {qty_rounded} below min {min_amount}, skipping order.")
            return 0.0
        
        return float(qty_rounded)
    
    except Exception as e:
        print(f"[ROUND_AMOUNT_QTY ERROR] {e}")
        # fallback: simple round with 6 decimals
        return float(round(qty, 6))
        
"""def round_amount_qty(qty, symbol_local=None):
    return float(round(qty, 6))
        """

def round_price(price):
    prec = MARKET_INFO.get('precision', {}).get('price')
    if prec is None:
        return price
    prec = int(prec)
    return float(round(price, prec))


In [10]:
def place_market_order_ccxt(side, amount, pos_side=None, reduce_only=False, symbol_local=None):
    """
    Unified market order helper for futures (demo/live) in hedge mode.
        side: 'buy' or 'sell' (action)
        amount: already-calculated asset amount (float)
        pos_side: 'LONG' or 'SHORT' (required in hedge mode)
        reduce_only: boolean; include reduceOnly param only when True
        symbol_local: optional symbol string (e.g., "BTC/USDT") to use for precision rounding
    Returns a dict-like result (never None) with keys including 'id','amount','status','error' on failure.
    """
    symbol_send = symbol_local if symbol_local else symbol  # use global symbol fallback
    amount = round_amount_qty(amount, symbol_send)
    if amount <= 0:
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': 'amount<=0 after rounding'}

    params = {}
    # Always include positionSide in hedge mode
    if pos_side:
        params['positionSide'] = pos_side

    # Only add reduceOnly if explicitly requested
    if reduce_only:
        params['reduceOnly'] = True

    try:
        # Market order: price=None
        order = exchange.create_order(symbol_send, 'market', side, amount, None, params)
        # Normalize response
        return {
            'id': order.get('id') if isinstance(order, dict) else getattr(order, 'id', None),
            'average': float(order.get('average', 0)) if isinstance(order, dict) else float(getattr(order, 'average', 0)),
            'amount': float(order.get('filled', order.get('amount', amount))) if isinstance(order, dict) else amount,
            'status': order.get('status', 'closed') if isinstance(order, dict) else 'closed',
            'raw': order
        }
    except Exception as e:
        print(f"❌ Order placement failed: {e}")
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': str(e)}

In [11]:
def compute_trade_fee(entry_price, qty):
    if FEE_MODE == "flat":
        return FLAT_FEE_USDT
    else:
        notional = entry_price * qty
        return notional * TRADING_FEE_RATE * 2


In [12]:
def place_grid(grid_direction, center_price, mode='trend', fraction=position_fraction):
    """
    Symmetric ladder grid system with fee-aware TP/SL:
    - For 'up' we create long grids (buy low, sell higher)
    - For 'down' we create short grids (sell high, buy lower)
    """
    gid = str(uuid.uuid4())[:8]
    grids[gid] = {
        'id': gid,
        'direction': grid_direction,
        'active': True,
        'created_at': datetime.utcnow(),
        'orders': [],
        'frozen': False
    }

    size_usd = 200
    half = grid_levels // 2
    grid_prices = [center_price + i * user_grid_spacing for i in range(-half, half + 1)]
    grid_prices = sorted(set(grid_prices))

    order_side = 'sell' if grid_direction == 'down' else 'buy'

    for i, level_price in enumerate(grid_prices):
        # --- Compute fee for this grid level ---
        qty = size_usd / level_price
        fee_cost = compute_trade_fee(level_price, qty)

        # Convert fee into price difference
        fee_price_diff = fee_cost / qty

        # --- Compute TP and SL with fee buffer ---
        if grid_direction == 'up':
            if i + 1 < len(grid_prices):
                tp_price = grid_prices[i + 1]
            else:
                tp_price = level_price * (1 + min_tp_pct)

            # Add buffer so TP covers trading fee
            tp_price += fee_price_diff * 1.5  # 1.5× fee buffer for net profit

            sl_price = level_price - 0.5 * user_grid_spacing
        else:  # grid_direction == 'down'
            if i > 0:
                tp_price = grid_prices[i - 1]
            else:
                tp_price = level_price * (1 - min_tp_pct)

            # Subtract buffer for fee
            tp_price -= fee_price_diff * 1.5

            sl_price = level_price + 0.5 * user_grid_spacing

        oid = str(uuid.uuid4())[:10]
        order = {
            'id': oid,
            'grid_id': gid,
            'side': order_side,
            'price': float(level_price),
            'size_usd': float(size_usd),
            'status': 'open',
            'tp': float(tp_price),
            'sl': float(sl_price)
        }
        open_orders.append(order)
        grids[gid]['orders'].append(oid)

    # --- Debug log for clarity ---
    print(f"[GRID PLACED] id={gid}, dir={grid_direction}, mode={mode}, levels={len(grid_prices)}")
    print("------------------------------------------------------------")
    for i, level_price in enumerate(grid_prices):
        order = grids[gid]['orders'][i]
        print(f"Grid {i+1}/{len(grid_prices)} | Price: {level_price:.2f} | TP: {order if isinstance(order, dict) else 'N/A'}")
    print("------------------------------------------------------------")

    return gid


In [13]:
from datetime import datetime, timezone

def block_zone_after_profit_drop(grid_id, price, side):
    """Block grid when profit drops by >= threshold."""
    already_blocked = any(b['grid_id'] == grid_id and b['side'] == side and b['active'] for b in profit_drop_blocked_zones)
    if not already_blocked:
        profit_drop_blocked_zones.append({
            'grid_id': grid_id,
            'price': price,
            'side': side,
            'blocked_at': datetime.now(timezone.utc),
            'active': True
        })
        print(f"🚫 [BLOCKED PROFIT DROP] {side.upper()} grid={grid_id} at price={price:.2f}")


def unblock_profit_drop_zones(current_price):
    """Unblock grids when price moves back beyond drop zone."""
    for b in profit_drop_blocked_zones[:]:
        if not b['active']:
            continue
        if b['side'] == 'long' and current_price > b['price']:
            b['active'] = False
            print(f"✅ [UNBLOCKED PROFIT DROP] LONG grid={b['grid_id']} (price recovered above {b['price']:.2f})")
        elif b['side'] == 'short' and current_price < b['price']:
            b['active'] = False
            print(f"✅ [UNBLOCKED PROFIT DROP] SHORT grid={b['grid_id']} (price recovered below {b['price']:.2f})")


In [14]:
from datetime import datetime, timezone

def block_zone_after_tp_reversal(grid_id, price, side):
    """Block grid when price reverses after TP."""
    already_blocked = any(b['grid_id'] == grid_id and b['side'] == side and b['active']
                          for b in tp_reversal_blocked_zones)
    if not already_blocked:
        tp_reversal_blocked_zones.append({
            'grid_id': grid_id,
            'price': price,
            'side': side,
            'blocked_at': datetime.now(timezone.utc),
            'active': True
        })
        print(f"🚫 [TP REVERSAL BLOCKED] {side.upper()} grid={grid_id} at {price:.2f}")

def unblock_tp_reversal_zones(current_price):
    """Unblock grids when price recovers beyond the TP reversal block level."""
    for b in tp_reversal_blocked_zones:
        if not b['active']:
            continue

        if b['side'] == 'long' and current_price > b['price']:
            b['active'] = False
            print(f"✅ [TP REVERSAL UNBLOCKED] LONG grid={b['grid_id']}")
        elif b['side'] == 'short' and current_price < b['price']:
            b['active'] = False
            print(f"✅ [TP REVERSAL UNBLOCKED] SHORT grid={b['grid_id']}")


In [15]:
def check_tp_reversal(last_price, threshold=0.4):
    """
    Detects take-profit reversals based on balance (PnL) drop instead of raw price movement.
    threshold = USDT loss amount to trigger a reversal (e.g., 0.2 means a $0.2 loss)
    """
    for zone in recent_tp_zones:
        if not zone['active']:
            continue

        tp_price = zone['tp_price']
        side = zone['side']
        grid_id = zone['grid_id']
        qty = zone['qty'] # make sure your TP zone stores quantity

        if qty <= 0:
            print(f"⚠️ [SKIP] grid={grid_id} has invalid qty={qty}")
            continue

        # Convert balance-based loss threshold into equivalent price difference
        price_threshold = threshold / qty

        if side == 'long' and last_price <= tp_price - price_threshold:
            block_zone_after_tp_reversal(grid_id, last_price, 'long')
            zone['active'] = False
            print(f"📉 [TP REVERSAL DETECTED] LONG grid={grid_id} "
                  f"(fell {threshold:.2f} USDT ≈ {price_threshold:.5f} price units below TP)")
        
        elif side == 'short' and last_price >= tp_price + price_threshold:
            block_zone_after_tp_reversal(grid_id, last_price, 'short')
            zone['active'] = False
            print(f"📈 [TP REVERSAL DETECTED] SHORT grid={grid_id} "
                  f"(rose {threshold:.2f} USDT ≈ {price_threshold:.5f} price units above TP)")


In [16]:
def handle_tp_reversal_protection(current_price, threshold=0.4):
    """
    Close any open position if unrealized PnL drops by a given USDT threshold
    after reaching its TP zone.
    
    threshold = USDT loss to trigger a reversal close (e.g., 0.2 means close if profit drops by $0.20)
    """
    for tp_zone in list(recent_tp_zones):
        if not tp_zone['active']:
            continue

        grid_id = tp_zone['grid_id']
        tp_price = tp_zone['tp_price']
        side = tp_zone['side']
        qty = tp_zone['qty']  # make sure your TP zone stores quantity

        if qty <= 0:
            print(f"⚠️ [SKIP TP REVERSAL] grid={grid_id} invalid qty={qty}")
            continue

        # Convert USDT loss threshold into equivalent price movement
        price_threshold = threshold / qty

        for pos in list(open_positions):
            if pos['grid_id'] != grid_id or pos['side'] != side:
                continue

            amount = pos['qty']
            if amount <= 0:
                continue

            # ✅ Check for reversal using balance-based (USDT) logic
            if side == 'long' and current_price <= tp_price - price_threshold:
                close_pos = place_market_order_ccxt(side='sell', amount=amount, pos_side='LONG')
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                tp_zone['active'] = False
                block_zone_after_tp_reversal(grid_id, current_price, side)
                print(f"🚨 [CLOSE TP REVERSAL LONG] grid={grid_id} closed at {current_price:.2f} "
                      f"(fell {threshold:.2f} USDT ≈ {price_threshold:.5f} below TP {tp_price:.2f})")

            elif side == 'short' and current_price >= tp_price + price_threshold:
                close_pos = place_market_order_ccxt(side='buy', amount=amount, pos_side='SHORT')
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                tp_zone['active'] = False
                block_zone_after_tp_reversal(grid_id, current_price, side)
                print(f"🚨 [CLOSE TP REVERSAL SHORT] grid={grid_id} closed at {current_price:.2f} "
                      f"(rose {threshold:.2f} USDT ≈ {price_threshold:.5f} above TP {tp_price:.2f})")


In [17]:
TRADE_FEE = 0.0009  # 0.09% per trade per side

def try_resolve_positions_by_tp_sl(current_price, profit_drop_threshold=0.8):
    """
    Close positions if TP/SL reached or if profit drops below threshold.
    Includes trading fee deduction.
    """
    closed = []

    for pos in list(open_positions):
        amount = pos.get('qty', 0)
        if amount <= 0:
            continue

        tp = pos.get('tp_price')
        sl = pos.get('sl_price')
        side = pos['side']
        grid_id = pos['grid_id']
        entry = pos['entry']

        # Calculate current profit before fees
        if side == 'long':
            profit = (current_price - entry) * amount
        else:
            profit = (entry - current_price) * amount

        # --- Apply estimated trading fees (entry + exit) ---
          # entry + exit
        net_profit = profit - 0.0472 * 2

        # --- 1. Close by TP/SL ---
        if side == 'long':
            if tp is not None and current_price >= tp:
                place_market_order_ccxt(side='sell', amount=amount, pos_side='LONG')
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'TP', current_price, net_profit))
                recent_tp_zones.append({
                    'grid_id': pos['grid_id'],
                    'tp_price': current_price,
                    'side': pos['side'],
                    'qty': amount,
                    'active': True
                })
                print(f"[CLOSE TP LONG] pos={pos['id']} closed at {current_price:.2f} | Net Profit: {net_profit:.4f} USDT")
                continue

        elif side == 'short':
            if tp is not None and current_price <= tp:
                place_market_order_ccxt(side='buy', amount=amount, pos_side='SHORT')
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'TP_SHORT', current_price, net_profit))
                recent_tp_zones.append({
                    'grid_id': pos['grid_id'],
                    'tp_price': current_price,
                    'side': pos['side'],
                    'qty': amount,
                    'active': True
                })
                print(f"[CLOSE TP SHORT] pos={pos['id']} closed at {current_price:.2f} | Net Profit: {net_profit:.4f} USDT")
                continue

        # --- 2. Close by profit drop threshold ---
        if 'peak_profit' not in pos:
            
            pos['peak_profit'] = net_profit
        else:
            if net_profit > pos['peak_profit']:
                pos['peak_profit'] = net_profit
            elif pos['peak_profit'] - net_profit >= profit_drop_threshold:
                if side == 'long':
                    place_market_order_ccxt(side='sell', amount=amount, pos_side='LONG')
                    print(f"[CLOSE PROFIT DROP LONG] pos={pos['id']} closed at {current_price:.2f} | Net Profit: {net_profit:.4f} USDT")
                else:
                    place_market_order_ccxt(side='buy', amount=amount, pos_side='SHORT')
                    print(f"[CLOSE PROFIT DROP SHORT] pos={pos['id']} closed at {current_price:.2f} | Net Profit: {net_profit:.4f} USDT")

                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'PROFIT_DROP', current_price, net_profit))
                block_zone_after_profit_drop(grid_id, current_price, side)

    return closed


In [18]:
def is_profit_drop_blocked(grid_id, side):
    """Check if the grid is currently blocked due to profit drop."""
    return any(b['grid_id'] == grid_id and b['side'] == side and b['active'] for b in profit_drop_blocked_zones)


In [19]:
def is_tp_reversal_blocked(grid_id, side):
    """Check if the grid is currently blocked due to TP reversal."""
    return any(
        b['grid_id'] == grid_id and b['side'] == side and b['active']
        for b in tp_reversal_blocked_zones
    )


In [20]:
def open_long(current_price, amount_usd):
    global balance
    filled = []

    for o in list(open_orders):
        if o['status'] != 'open' or o['side'] != 'buy':
            continue

        grid_id = o['grid_id']
        order_entry_price = o['price']

        if is_profit_drop_blocked(grid_id, 'long') or is_tp_reversal_blocked(grid_id, 'long'):
            continue
        if grid_id in has_open_position_zones:
            continue

        grid_orders = [oo for oo in open_orders if oo['grid_id'] == grid_id and oo['side'] == 'buy']
        grid_prices = sorted([oo['price'] for oo in grid_orders])
        i = grid_prices.index(o['price'])
        lower_grid_price = grid_prices[i - 1] if i > 0 else None

        # Price must fall into zone
        if not ((lower_grid_price or 0) < current_price <= o['price']):
            continue

        size_usd = amount_usd
        if size_usd <= 0:
            print("❌ Not enough balance to open LONG.")
            continue

        amount = size_usd / current_price
        ccxt_order = place_market_order_ccxt(side='buy', amount=amount, pos_side='LONG')

        entry_price = ccxt_order.get('average', current_price)
        qty = ccxt_order.get('amount', amount)
        status = ccxt_order.get('status')

        # --- Fee-aware TP/SL ---
        fee_cost = compute_trade_fee(entry_price, qty)
        fee_price_diff = fee_cost / qty

        tp_price = entry_price + user_grid_spacing + fee_price_diff * 1.5
        sl_price = entry_price - 0.5 * user_grid_spacing - fee_price_diff

        pos = {
            'id': str(uuid.uuid4())[:10],
            'side': 'long',
            'entry': float(entry_price),
            'order_entry_price': float(order_entry_price),
            'ccxt_id': ccxt_order.get('id'),
            'qty': qty,
            'notional': size_usd,
            'grid_id': grid_id,
            'created_at': datetime.now(timezone.utc),
            'last_price': current_price,
            'active': True,
            'tp_price': float(tp_price),
            'sl_price': float(sl_price)
        }

        open_positions.append(pos)
        has_open_position_zones.add(grid_id)

        print(f"[LONG FILLED] status={status} entry={entry_price:.2f} qty={qty:.6f} TP={tp_price:.2f} SL={sl_price:.2f}")
        filled.append(o)

    return filled


def open_short(current_price, amount_usd):
    global balance
    filled = []

    for o in list(open_orders):
        if o['status'] != 'open' or o['side'] != 'sell':
            continue

        grid_id = o['grid_id']
        order_entry_price = o['price']

        if is_profit_drop_blocked(grid_id, 'short') or is_tp_reversal_blocked(grid_id, 'short'):
            continue
        if grid_id in has_open_position_zones:
            continue

        grid_orders = [oo for oo in open_orders if oo['grid_id'] == grid_id and oo['side'] == 'sell']
        grid_prices = sorted([oo['price'] for oo in grid_orders])
        i = grid_prices.index(o['price'])
        upper_grid_price = grid_prices[i + 1] if i + 1 < len(grid_prices) else None

        if not ((upper_grid_price > current_price >= o['price']) if upper_grid_price else (current_price >= o['price'])):
            continue

        size_usd = amount_usd
        if size_usd <= 0:
            print("❌ Not enough balance to open SHORT.")
            continue

        amount = size_usd / current_price
        ccxt_order = place_market_order_ccxt(side='sell', amount=amount, pos_side='SHORT')

        entry_price = ccxt_order.get('average', current_price)
        qty = ccxt_order.get('amount', amount)
        status = ccxt_order.get('status')

        # --- Fee-aware TP/SL ---
        fee_cost = compute_trade_fee(entry_price, qty)
        fee_price_diff = fee_cost / qty

        tp_price = entry_price - user_grid_spacing - fee_price_diff * 1.5
        sl_price = entry_price + 0.5 * user_grid_spacing + fee_price_diff

        pos = {
            'id': str(uuid.uuid4())[:10],
            'side': 'short',
            'entry': float(entry_price),
            'order_entry_price': float(order_entry_price),
            'ccxt_id': ccxt_order.get('id'),
            'qty': qty,
            'notional': size_usd,
            'grid_id': grid_id,
            'created_at': datetime.now(timezone.utc),
            'last_price': current_price,
            'active': True,
            'tp_price': float(tp_price),
            'sl_price': float(sl_price)
            
        }

        open_positions.append(pos)
        has_open_position_zones.add(grid_id)

        print(f"[SHORT FILLED] status={status} entry={entry_price:.2f} qty={qty:.6f} TP={tp_price:.2f} SL={sl_price:.2f}")
        filled.append(o)

    return filled


In [21]:
HEDGE_MODE = None  # None => try to auto-detect, True/False => force

def detect_hedge_mode():
    """
    Try to detect whether the futures account is in hedge (dual-side) mode.
    If detection fails, return False (safe default).
    """
    global HEDGE_MODE
    if HEDGE_MODE is not None:
        return bool(HEDGE_MODE)
    try:
        # CCXT sometimes exposes this raw endpoint; try a couple of possibilities
        # If these calls fail, we fallback to False (one-way mode).
        try:
            # newer ccxt: fapiPrivate_get_positionSide_dual or fapiPrivateGetPositionSideDual
            # try both spellings
            r = getattr(exchange, 'fapiPrivate_get_positionSide_dual', None)
            if callable(r):
                resp = r()
                val = resp.get('dualSidePosition') if isinstance(resp, dict) else None
                if val is not None:
                    HEDGE_MODE = bool(val)
                    return HEDGE_MODE
        except Exception:
            pass

        # fallback try fetch_positions and look for 'positionSide' in info
        try:
            positions = exchange.fetch_positions()
            for p in positions:
                info = p.get('info') if isinstance(p, dict) else None
                if info and ('positionSide' in info or 'positionSide' in p):
                    HEDGE_MODE = True
                    return True
        except Exception:
            pass

    except Exception:
        pass

    # default safe: assume one-way mode
    HEDGE_MODE = False
    return False

In [22]:
def close_all_positions(current_price):
    global open_positions
    # attempt to auto-detect mode
    hedge = detect_hedge_mode()
    for p in list(open_positions):
        exit_price = current_price
        side = 'sell' if p['side'] == 'long' else 'buy'
        pos_side = None
        if hedge:
            pos_side = 'LONG' if p['side'] == 'long' else 'SHORT'
        print(f"[CLOSE ATTEMPT] id={p['id']} side={p['side']} qty={p['qty']:.8f} hedge={hedge} pos_side={pos_side}")
        res = place_market_order_ccxt(side, p['qty'], pos_side=pos_side, symbol_local=p.get('symbol', symbol))
        print(f"[CLOSE RESULT] {res}")
        # remove if closed or 'closed' in status
        if res and res.get('status') in ('closed', 'ok', 'filled') or res.get('id'):
            try:
                open_positions.remove(p)
            except ValueError:
                pass
        else:
            # if failed, log and continue (do not remove)
            print(f"[WARN] Forced close failed for pos {p['id']}: {res.get('error')}")


In [23]:
def cancel_grid_orders_ccxt(gid):
    global open_orders
    to_cancel = [o for o in open_orders if o['grid_id'] == gid]
    for o in to_cancel:
        id = o.get('id')
        if id:
            open_orders.remove(o)
    print(f"[GRID CANCELLED] id={gid}, canceled_orders={len(to_cancel)}")
    grids.pop(gid, None)

In [24]:
from datetime import datetime, timezone

def block_zone_after_profit_drop(grid_id, price, side):
    """Block grid when profit drops by >= threshold."""
    already_blocked = any(b['grid_id'] == grid_id and b['side'] == side and b['active'] for b in profit_drop_blocked_zones)
    if not already_blocked:
        profit_drop_blocked_zones.append({
            'grid_id': grid_id,
            'price': price,
            'side': side,
            'blocked_at': datetime.now(timezone.utc),
            'active': True
        })
        print(f"🚫 [BLOCKED PROFIT DROP] {side.upper()} grid={grid_id} at price={price:.2f}")


def unblock_profit_drop_zones(current_price):
    """Unblock grids when price moves back beyond drop zone."""
    for b in profit_drop_blocked_zones[:]:
        if not b['active']:
            continue
        if b['side'] == 'long' and current_price > b['price']:
            b['active'] = False
            print(f"✅ [UNBLOCKED PROFIT DROP] LONG grid={b['grid_id']} (price recovered above {b['price']:.2f})")
        elif b['side'] == 'short' and current_price < b['price']:
            b['active'] = False
            print(f"✅ [UNBLOCKED PROFIT DROP] SHORT grid={b['grid_id']} (price recovered below {b['price']:.2f})")


In [25]:
wait_for_reversal = False
global balance, open_positions, open_orders, grids
last_prediction = None
last_prediction_time = None
last_trend_direction = None


# Warm-up: compute last reversal
back_df = fetch_latest_ohlcv(symbol, model_timeframe, 120 + window_size)
feat_df = compute_indicators(back_df).dropna()

preds = [
predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
for i in range(window_size, len(feat_df))
]
last_reversal = next((p for p in reversed(preds) if p != 1), None)
last_prediction = last_reversal if last_reversal is not None else 1
print(f"[INIT] last_pred={last_prediction} ({'DOWN' if last_prediction==0 else 'HOLD' if last_prediction==1 else 'UP'})")

last_price = float(exchange.fetch_ticker(symbol)['last'])
user_grid_spacing =  10             #get_current_atr() * atr_multiplier
print(f"[GRID SPACING] initial ATR-based spacing = {user_grid_spacing:.2f}")

if last_prediction == 0:
    place_grid('down', last_price, mode='trend')
    last_trend_direction = 'short'
elif last_prediction == 2:
    place_grid('up', last_price, mode='trend')
    last_trend_direction = 'long'
else:
    print("Waiting for clear trend signal to start grid...")  
    
for order in open_orders:
    print(f"Order ID: {order['id']}, Price: {order['price']}")
  





[INIT] last_pred=2 (UP)
[GRID SPACING] initial ATR-based spacing = 10.00
[GRID PLACED] id=05446138, dir=up, mode=trend, levels=17
------------------------------------------------------------
Grid 1/17 | Price: 105125.10 | TP: N/A
Grid 2/17 | Price: 105135.10 | TP: N/A
Grid 3/17 | Price: 105145.10 | TP: N/A
Grid 4/17 | Price: 105155.10 | TP: N/A
Grid 5/17 | Price: 105165.10 | TP: N/A
Grid 6/17 | Price: 105175.10 | TP: N/A
Grid 7/17 | Price: 105185.10 | TP: N/A
Grid 8/17 | Price: 105195.10 | TP: N/A
Grid 9/17 | Price: 105205.10 | TP: N/A
Grid 10/17 | Price: 105215.10 | TP: N/A
Grid 11/17 | Price: 105225.10 | TP: N/A
Grid 12/17 | Price: 105235.10 | TP: N/A
Grid 13/17 | Price: 105245.10 | TP: N/A
Grid 14/17 | Price: 105255.10 | TP: N/A
Grid 15/17 | Price: 105265.10 | TP: N/A
Grid 16/17 | Price: 105275.10 | TP: N/A
Grid 17/17 | Price: 105285.10 | TP: N/A
------------------------------------------------------------
Order ID: e200d802-c, Price: 105125.1
Order ID: 7db4c4f7-7, Price: 105135.1
O

/tmp/ipykernel_6631/4008296404.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'created_at': datetime.utcnow(),


In [26]:
monitoring_reversal_long = False
monitoring_reversal_short = False
current_pred = last_prediction
last_trend_direction = 'long' if last_prediction == 2 else 'short' if last_prediction == 0 else None
while True:
    last_price = float(exchange.fetch_ticker(symbol)['last'])
    trade_usd = 100
    profit_drop_threshold = 1.2  # USDT loss before confirming reversal

    # 🧠 --- 1. Make a fresh ML prediction each loop ---
    df_feat = fetch_latest_ohlcv(symbol, model_timeframe, limit)
    feat_df = compute_indicators(df_feat).dropna()
    if len(feat_df) >= window_size:
        window = feat_df[FEATURES].values[-window_size:]
        new_pred = predict_label_from_window(window)
        current_pred = new_pred
        print(f"[{datetime.now(timezone.utc)}] ML pred={new_pred}")
    else:
        current_pred = last_prediction  # fallback if not enough data
        
    if current_pred != last_prediction:
        print(f"[ML PREDICTION CHANGE] {last_prediction} → {current_pred} ")
        last_prediction = current_pred
    else:
        print(f"[ML PREDICTION] remains {current_pred}")       


    """# --- 3. Unblock and resolve ---
    unblock_profit_drop_zones(last_price)
    check_tp_reversal(last_price, threshold=0.8)
    unblock_tp_reversal_zones(last_price)
    closed = try_resolve_positions_by_tp_sl(last_price, profit_drop_threshold)
    closed_tp_reversal = handle_tp_reversal_protection(last_price, threshold=0.8)

    # --- 4. Handle possible reversals with confirmation ---
    if last_trend_direction == 'long':
        # Only trigger once — when ML first warns us of a downtrend
        if not monitoring_reversal_long and current_pred == 0:
            monitoring_reversal_long = True
            print("[⚠️ POSSIBLE REVERSAL] LONG → SHORT detected, start monitoring loss...")

        # Once monitoring starts, continue even if ML no longer says 0
        if monitoring_reversal_long:
            open_long(last_price, trade_usd)

            total_long_loss = 0.0
            for pos in list(open_positions):
                amount = pos.get('qty', 0)
                side = pos['side']
                grid_id = pos['grid_id']
                entry = pos['entry']

                entry_fee = compute_trade_fee(entry, amount)
                exit_fee = compute_trade_fee(last_price, amount)
                total_fees = entry_fee + exit_fee

                if side == 'long':
                    gross_profit = (last_price - entry) * amount
                else:
                    gross_profit = (entry - last_price) * amount

                profit = gross_profit - total_fees

                if 'peak_profit_2' not in pos:
                    pos['peak_profit_2'] = profit
                else:
                    if profit > pos['peak_profit_2']:
                        pos['peak_profit_2'] = profit

                total_long_loss += (pos['peak_profit_2'] - profit)

            # If loss exceeds threshold, confirm reversal
            if total_long_loss >= profit_drop_threshold:
                print(f"🚨 [CONFIRMED REVERSAL] Long loss {total_long_loss:.2f} ≥ {profit_drop_threshold}. Switching to SHORT.")
                close_all_positions(last_price)
                for gid in list(grids.keys()):
                    cancel_grid_orders_ccxt(gid)
                    grids.pop(gid, None)
                place_grid('down', last_price, mode='trend')
                last_trend_direction = 'short'
                monitoring_reversal_long = False

    elif last_trend_direction == 'short':
        # Only trigger once — when ML first warns us of an uptrend
        if not monitoring_reversal_short and current_pred == 2:
            monitoring_reversal_short = True
            print("[⚠️ POSSIBLE REVERSAL] SHORT → LONG detected, start monitoring loss...")

        # Once monitoring starts, continue even if ML no longer says 2
        if monitoring_reversal_short:
            open_short(last_price, trade_usd)

            total_short_loss = 0.0
            for pos in list(open_positions):
                amount = pos.get('qty', 0)
                side = pos['side']
                grid_id = pos['grid_id']
                entry = pos['entry']

                entry_fee = compute_trade_fee(entry, amount)
                exit_fee = compute_trade_fee(last_price, amount)
                total_fees = entry_fee + exit_fee

                if side == 'long':
                    gross_profit = (last_price - entry) * amount
                else:
                    gross_profit = (entry - last_price) * amount

                profit = gross_profit - total_fees

                if 'peak_profit_2' not in pos:
                    pos['peak_profit_2'] = profit
                else:
                    if profit > pos['peak_profit_2']:
                        pos['peak_profit_2'] = profit

                total_short_loss += (pos['peak_profit_2'] - profit)

            # If loss exceeds threshold, confirm reversal
            if total_short_loss >= profit_drop_threshold:
                print(f"🚨 [CONFIRMED REVERSAL] Short loss {total_short_loss:.2f} ≥ {profit_drop_threshold}. Switching to LONG.")
                close_all_positions(last_price)
                for gid in list(grids.keys()):
                    cancel_grid_orders_ccxt(gid)
                    grids.pop(gid, None)
                place_grid('up', last_price, mode='trend')
                last_trend_direction = 'long'
                monitoring_reversal_short = False


    
    # --- 5. Continue in current direction ---
    if current_pred == 0:
        open_short(last_price, trade_usd)
        last_prediction = 0
    elif current_pred == 2:
        last_prediction = 2
        open_long(last_price, trade_usd)
        
    elif current_pred == 1 and last_trend_direction == 'long':
        open_long(last_price, trade_usd)
    elif current_pred == 1 and last_trend_direction == 'short':
        open_short(last_price, trade_usd)
            

    # --- 6. Debug info ---
    for pos in open_positions:
        print(f"Open Position ID: {pos['id']} Side: {pos['side']} Entry: {pos['entry']:.2f} Qty: {pos['qty']:.6f} Peak Profit: {pos.get('peak_profit', 0):.6f}")
    print(f"Total Open Positions: {len(open_positions)}, Closed this loop: {len(closed)}")"""

    time.sleep(1)


[2025-11-11 07:00:05.278494+00:00] ML pred=1
[ML PREDICTION CHANGE] 2 → 1 
[2025-11-11 07:00:07.096650+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:08.929892+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:10.765640+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:12.649139+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:15.468296+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:17.756878+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:19.603398+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:21.442568+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:23.296722+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:26.399393+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:28.227747+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:30.061446+00:00] ML pred=1
[ML PREDICTION] remains 1
[2025-11-11 07:00:31.901696+00:00] ML pred=1
[ML PREDICTION] remains 1
[2

KeyboardInterrupt: 